In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

In [ ]:
import os 
import pandas as pd
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker

if not pt.started():
  pt.init()

gdrive_path = 'drive/MyDrive/timo'
batch_size = 4
monoT5 = MonoT5ReRanker(batch_size=batch_size)

B = [round(0.2 + 0.05 * i, 3) for i in range(0,13)]
K1 = [round(0.5 + 0.05 * i, 3) for i in range(0,13)]

for b in B:
    for k1 in K1:
        bm25_file = 'bm25_{}_{}.csv'.format(str(b), str(k1))
        csv_path = os.path.join(gdrive_path, 'csv', bm25_file)
        run = pd.read_csv(csv_path)
        run_monoT5 = monoT5.transform(run)
        run_monoT5 = run_monoT5.sort_values(['qid','score'], ascending=False).groupby('qid').head(1000)
        monoT5_file = 'monoT5_{}_bm25_{}_{}.txt'.format(str(batch_size), str(b), str(k1))
        monoT5_path = os.path.join(gdrive_path, 'monoT5', monoT5_file)
        pt.io.write_results(run_monoT5, monoT5_path, format='trec',run_name=monoT5_file[:-4])